In [ ]:
%%capture
%config Completer.use_jedi = False
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

# Install on Google Colab
import subprocess
import sys

from IPython import get_ipython

install_packages = "google.colab" in str(get_ipython())
if install_packages:
    for package in ["ampform[doc]", "graphviz"]:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", package]
        )

In [ ]:
import logging
import warnings

logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)

warnings.filterwarnings("ignore")

# Analytic continuation

Analytic continuation allows one to handle resonances just below threshold ($m_0 < m_a + m_b$  in Eq. {eq}`relativistic_breit_wigner_with_ff`). In practice, this entails using a specific function for $\rho$ in Eq. {eq}`coupled_width`.

## Definitions

Three usual choices for $\rho$ are the following:

In [ ]:
import sympy as sp
from IPython.display import Math

1. The {func}`~sympy.functions.elementary.miscellaneous.sqrt` or `.ComplexSqrt` of {func}`.breakup_momentum_squared`:

In [ ]:
from ampform.dynamics import breakup_momentum_squared

s, m_a, m_b = sp.symbols("s, m_a, m_b", real=True)
q_squared = breakup_momentum_squared(s, m_a, m_b)

In [ ]:
q_squared_symbol = sp.Symbol("q^{2}(s)")
Math(f"{sp.latex(q_squared_symbol)} = {sp.latex(q_squared)}")

2. The 'normal' {func}`.phase_space_factor` (the denominator makes the difference to {eq}`coupled_width`!):

In [ ]:
from ampform.dynamics import phase_space_factor

rho = phase_space_factor(s, m_a, m_b)

In [ ]:
rho_subs = rho.subs(4 * q_squared, 4 * q_squared_symbol)
Math(fR"\hat{{\rho}}(s) = {sp.latex(rho_subs)}")

3. The following 'case-by-case' **analytic continuation**, {func}`.phase_space_factor_ac`:

In [ ]:
from ampform.dynamics import phase_space_factor_ac

rho_analytic = phase_space_factor_ac(s, m_a, m_b)

In [ ]:
from ampform.dynamics import _analytic_continuation

rho_hat_symbol = sp.Symbol(R"\hat{\rho}")
_analytic_continuation(rho_hat_symbol, s, s_threshold=(m_a + m_b) ** 2)

with

:::{margin}

Mind the absolute value!

:::

In [ ]:
from ampform.dynamics import _phase_space_factor_hat

rho_hat = _phase_space_factor_hat(s, m_a, m_b)
rho_hat_subs = rho_hat.subs(4 * q_squared, 4 * q_squared_symbol)
Math(f"{sp.latex(rho_hat_symbol)} = {sp.latex(rho_hat_subs)}")

## Visualization

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import mpl_interactions.ipyplot as iplt
import numpy as np

import symplot

In [ ]:
m = sp.Symbol("m", real=True)
rho = phase_space_factor(m ** 2, m_a, m_b)
rho_ac = phase_space_factor_ac(m ** 2, m_a, m_b)
np_rho, sliders = symplot.prepare_sliders(plot_symbol=m, expression=rho)
np_rho_ac = sp.lambdify((m, m_a, m_b), rho_ac, "numpy")

In [ ]:
plot_domain = np.linspace(0, 3, 1_000)
sliders.set_ranges(
    m_a=(0, 2, 200),
    m_b=(0, 2, 200),
)
sliders.set_values(
    m_a=0.8,
    m_b=1.25,
)

In [ ]:
from ampform.dynamics import ComplexSqrt

np_rho, sliders = symplot.prepare_sliders(plot_symbol=m, expression=rho)
np_rho_ac = sp.lambdify((m, m_a, m_b), rho_ac, "numpy")
np_breakup_momentum = sp.lambdify(
    (m, m_a, m_b), ComplexSqrt(q_squared.subs(s, m ** 2)), "numpy"
)

plot_domain = np.linspace(0, 3, 500)
sliders.set_ranges(
    m_a=(0, 2, 200),
    m_b=(0, 2, 200),
)
sliders.set_values(
    m_a=0.6,
    m_b=1.25,
)

fig, axes = plt.subplots(1, 3, figsize=[10, 4], tight_layout=True)
fig.canvas.toolbar_visible = False

ax_q, ax_rho, ax_rho_ac = axes
for ax in axes:
    ax.set_xlabel("$m$")

ylim_q = (-0.04, 1.0)
ylim_rho = (-0.002, 0.05)


def func_imag(func, *args, **kwargs):
    return lambda *args, **kwargs: func(*args, **kwargs).imag


def func_real(func, *args, **kwargs):
    return lambda *args, **kwargs: func(*args, **kwargs).real


q_math = ComplexSqrt(q_squared_symbol)
ax_q.set_title(f"${sp.latex(q_math)}$")
controls = iplt.plot(
    plot_domain,
    func_real(np_breakup_momentum),
    label="real",
    **sliders,
    ylim=ylim_q,
    ax=ax_q,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_breakup_momentum),
    label="imaginary",
    controls=controls,
    ylim=ylim_q,
    ax=ax_q,
    alpha=0.7,
)

ax_rho.set_title(f"${sp.latex(rho_subs)}$")
iplt.plot(
    plot_domain,
    func_real(np_rho),
    label="real",
    controls=controls,
    ylim=ylim_rho,
    ax=ax_rho,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_rho),
    label="imaginary",
    controls=controls,
    ylim=ylim_rho,
    ax=ax_rho,
    alpha=0.7,
)

ax_rho_ac.set_title("analytic")
iplt.plot(
    plot_domain,
    func_real(np_rho_ac),
    label="real",
    controls=controls,
    ylim=ylim_rho,
    ax=ax_rho_ac,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_rho_ac),
    label="imaginary",
    controls=controls,
    ylim=ylim_rho,
    ax=ax_rho_ac,
    alpha=0.7,
)
plt.legend(loc="upper right")

plt.show()

{{ run_interactive }}

In [ ]:
if STATIC_WEB_PAGE:
    from IPython.display import SVG

    output_file = "analytic-continuation.svg"
    plt.savefig(output_file)
    display(SVG(output_file))